# Modeling

## Load the train data

In [1]:
import pandas as pd

train_df = pd.read_csv("../data/train.csv")

print(train_df.shape)
train_df.head()

(272, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,42.0,0.0,3.0,120.0,209.0,0.0,0.0,173.0,0.0,0.0,2.0,0.0,3.0,0
1,43.0,0.0,3.0,122.0,213.0,0.0,0.0,165.0,0.0,0.2,2.0,0.0,3.0,0
2,63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,1
3,58.0,1.0,4.0,125.0,300.0,0.0,2.0,171.0,0.0,0.0,1.0,2.0,7.0,1
4,41.0,0.0,2.0,126.0,306.0,0.0,0.0,163.0,0.0,0.0,1.0,0.0,3.0,0


## Exploratory Data Analysis

In [2]:
#!pip install pandas-profiling

In [3]:
from pandas_profiling import ProfileReport

In [4]:
profile = ProfileReport(train_df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Define the columns
cat_cols = ["sex", "cp", "fbs", "restecg", "exang", "slope", "ca", "thal"]
cont_cols = ["age", "trestbps", "chol", "thalach", "oldpeak"]

## Split Features and Response

In [6]:
X_train = train_df.drop("target", axis=1)
y_train = train_df["target"]

## Data Transformations

In [20]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [21]:
# One hot encode the categorical columns
ohe = OneHotEncoder(drop="first")

# Scale the continuous columns
sc = StandardScaler()

# Column transformer to apply transformations on both categorical and continuous columns
ct = ColumnTransformer([
    ("One Hot Encoding", ohe, cat_cols),
    ("Scaling", sc, cont_cols)
])

## ML Model

In [22]:
# KNN Model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

## Pipeline to combine feature engineering and ML model

In [23]:
# Sklearn pipeline
from sklearn.pipeline import Pipeline

pipeline_model = Pipeline([
    ("Data Transformations", ct),
    ("KNN Model", knn)
])

## Fit the Pipeline Model locally

In [24]:
pipeline_model.fit(X_train, y_train)

Pipeline(steps=[('Data Transformations',
                 ColumnTransformer(transformers=[('One Hot Encoding',
                                                  OneHotEncoder(drop='first'),
                                                  ['sex', 'cp', 'fbs',
                                                   'restecg', 'exang', 'slope',
                                                   'ca', 'thal']),
                                                 ('Scaling', StandardScaler(),
                                                  ['age', 'trestbps', 'chol',
                                                   'thalach', 'oldpeak'])])),
                ('KNN Model', KNeighborsClassifier())])

In [25]:
pipeline_model.score(X_train, y_train)

0.875

## Fit the Pipeline Model on Sagemaker!

## Deploy as Real Time Inference